In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

from jpasLAEs.load_jpas_catalogs import load_minijpas_jnep
from load_paus_cat import load_paus_cat

from astropy.coordinates import SkyCoord
import astropy.units as u

import numpy as np

In [ ]:
# Load miniJPAS
mj_cat_dir = '/home/alberto/cosmos/LAEs/csv'
mj_cat = load_minijpas_jnep(mj_cat_dir)

# Load PAUS
field_name = 'W3'
path_to_cat = f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_{field_name}.csv'
paus_cat = load_paus_cat(path_to_cat)

In [ ]:
# Xmatch
coords_paus = SkyCoord(ra=paus_cat['RA'] * u.deg,
                       dec=paus_cat['DEC'] * u.deg)
coords_mj = SkyCoord(ra=mj_cat['RA'] * u.deg,
                       dec=mj_cat['DEC'] * u.deg)
                    
xm_id, ang_dist, _= coords_paus.match_to_catalog_sky(coords_mj)

# Objects with 1 arcsec of separation
mask_dist = (ang_dist <= 1 * u.arcsec)

print(f'Matched: {sum(mask_dist)}')

In [ ]:
# Select LAEs
from LAE_selection_method import select_LAEs

nb_min, nb_max = 0, 20
r_min, r_max = 17, 24

paus_cat = select_LAEs(paus_cat, nb_min, nb_max, r_min, r_max,
                       ew0min_lya=30, ewmin_other=100)

In [ ]:
import pandas as pd

mj_sel_dir = '/home/alberto/cosmos/LAEs/csv/selection.csv'
mj_selection = pd.read_csv(mj_sel_dir)
mj_selection.head()

In [ ]:
from jpasLAEs.utils import z_NB as jpas_z_NB

# Add stuff to mj_cat
mj_cat['L_lya'] = np.zeros_like(mj_cat['number']).astype(float)
mj_cat['z_NB'] = np.zeros_like(mj_cat['number']).astype(float)
mj_cat['nice_lya'] = np.zeros_like(mj_cat['number']).astype(bool)

for i, src in enumerate(mj_selection['src']):
    mj_cat['L_lya'][src] = mj_selection['L_lya'][i]
    mj_cat['z_NB'][src] = jpas_z_NB(mj_selection['nb_sel'][i])
    mj_cat['nice_lya'][src] = True

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

L_bins = np.linspace(42, 46, 20)

ax.hist(paus_cat['L_lya'][paus_cat['nice_lya']],
        histtype='step', label='PAUS W3',
        density=True)
ax.hist(mj_cat['L_lya'][mj_cat['nice_lya']],
        histtype='step', label='miniJPAS',
        density=True)

ax.legend(fontsize=14)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.scatter(paus_cat['z_NB'], mj_cat['z_NB'][xm_id])

xx = [-100, 100]
ax.plot(xx, xx, ls='--', c='r')

ax.set_xlabel('Redshift (PAUS)')
ax.set_ylabel('Redshift (miniJPAS)')
ax.set_xlim(2, 5)
ax.set_ylim(2, 5)

plt.show()